#Try the course example: Minimal Character-level Vanilla RNN model

In [59]:
import numpy as np

In [60]:
data = open('input.txt','r').read()

In [61]:
chars = list(set(data[0:-1]))
data_size, vocab_size = len(data), len(chars)
print 'data has %d characters, %d unique.' % (data_size, vocab_size)

data has 118 characters, 26 unique.


In [62]:
char_to_ix = {ch:i for i, ch in enumerate(chars)}
ix_to_char = {i:ch for i, ch in enumerate(chars)}

In [63]:
#hidden parameters
hidden_size = 100 # number of neurons in hidden layer
seq_length = 10 # number of steps to unroll the RNN for
learning_rate = 1e-1

In [64]:
# model parameters
Wxh = np.random.randn(hidden_size, vocab_size) * 0.01 # input to hidden
Whh = np.random.randn(hidden_size, hidden_size) * 0.01 # hidden to hidden
Why = np.random.randn(vocab_size, hidden_size) * 0.01 # hidden to output
bh = np.zeros((hidden_size, 1)) # hidden bias
by = np.zeros((vocab_size, 1))

In [65]:
def lossFun(inputs, targets, hprev):
    """
    inputs targets are both list of integers
    hprev is Hx1 array of initial hidden state
    returns the loss, gradients on model parameters, and last hidden state
    """
    xs, hs, ys, ps = {},{},{},{}
    hs[-1] = np.copy(hprev)
    loss = 0
    # forward pass
    for t in xrange(len(inputs)):
        xs[t] = np.zeros((vocab_size, 1)) # encode in 1-of-k representation
        xs[t][inputs[t]] = 1
        hs[t] = np.tanh(np.dot(Wxh, xs[t]) + np.dot(Whh, hs[t-1]) + bh) # hidden state
        ys[t] = np.dot(Why, hs[t]) + by # unnormalized log probabilities for next chars
        ps[t] = np.exp(ys[t]) / np.sum(np.exp(ys[t]))
        loss += -np.log(ps[t][targets[t], 0]) # softmax(cross-entropy loss)
    # backward pass: compute gradients going backwards
    dWxh, dWhh, dWhy = np.zeros_like(Wxh), np.zeros_like(Whh), np.zeros_like(Why)
    dbh, dby = np.zeros_like(bh), np.zeros_like(by)
    dhnext = np.zeros_like(hs[0])
    for t in reversed(xrange(len(inputs))):
        dy = np.copy(ps[t])
        dy[targets[t]] -= 1
        dWhy += np.dot(dy, hs[t].T)
        dby += dy
        dh = np.dot(Why.T, dy) + dhnext # backprop into h
        dhraw = (1 - hs[t] * hs[t]) * dh # backprop through tanh nonlinearity
        dbh += dhraw
        dWxh += np.dot(dhraw, xs[t].T)
        dWhh += np.dot(dhraw, hs[t - 1].T)
        dhnext = np.dot(Whh.T, dhraw)
    for dparam in [dWxh, dWhh, dWhy, dbh, dby]:
        np.clip(dparam, -5, 5, out=dparam) # clip to mitigate exploding gradients
    return loss, dWxh, dWhh, dWhy, dbh, dby, hs[len(inputs) - 1]

In [66]:
def sample(h, seed_ix, n):
    """
    sample  a sequence of integers from the model
    h is memory state, seed_ix is seed letter for first time step
    """
    x = np.zeros((vocab_size, 1))
    x[seed_ix] = 1
    ixes = []
    for t in xrange(n):
        h = np.tanh(np.dot(Wxh, x) + np.dot(Whh, h) + bh)
        y = np.dot(Why, h) + by
        p = np.exp(y) / np.sum(np.exp(y))
        ix = np.random.choice(range(vocab_size), p = p.ravel())
        x = np.zeros((vocab_size, 1))
        x[ix] = 1
        ixes.append(ix)
    return ixes

In [68]:
n, p = 0, 0
mWxh, mWhh, mWhy = np.zeros_like(Wxh), np.zeros_like(Whh), np.zeros_like(Why)
mbh, mby = np.zeros_like(bh), np.zeros_like(by) # memory variabels for Adagrad
smooth_loss = - np.log(1.0/vocab_size) * seq_length
while True:
    # prepare inputs
    if p+seq_length+1 >= len(data) or n == 0:
        hprev = np.zeros((hidden_size, 1)) # reset RNN
        p = 0
    inputs = [char_to_ix[ch] for ch in data[p:p+seq_length]]
    targets = [char_to_ix[ch] for ch in data[p+1:p+seq_length+1]]
                         
    # sample from the model now and then
    if n % 100 == 0:
        sample_ix = sample(hprev, inputs[0], 200)
        txt = ''.join(ix_to_char[ix] for ix in sample_ix)
        print '____\n %s \n' % (txt, )
    
    # forward seq_length characters through the net and fetch gradient
    loss, dWxh, dWhh, dWhy, dbh, dby, hprev = lossFun(inputs, targets, hprev)
    smooth_loss = smooth_loss * 0.999 + loss * 0.001
    if n % 100 == 0: print 'iter %d, loss: %f' % (n, smooth_loss) # print progress
    
    # perform parameter update with Adagrad
    for param, dparam, mem in zip([Wxh, Whh, Why, bh, by],
                                  [dWxh, dWhh, dWhy, dbh, dby],
                                  [mWxh, mWhh, mWhy, mbh, mby]):
        mem += dparam * dparam
        param += -learning_rate * dparam / np.sqrt(mem + 1e-8) #adagrad update
    p += 1
    n += 1

____
 hewei Hu Zewei Hu Zexi Chen Shu Chen Shuai Yu Zexi Chen Shuai Yang Dong ang Deoliang Nie Zeyuan Chen Yuepeng Qi Yiwei Qi Yiwei Li LongMa Yanhong Li Sicong Wang Liang Liang DongMa Yanhong Li Sicong Wan 

iter 0, loss: 32.548474
____
 g LicSicon Yi Cicong ai Sicong Li Sicong LicSicong Liciconcong Li Yi Sicong Sicng Si ong Licncong Licng Li Sicong Si Sicoi Liconi Sicong Liw Li Sicong Li Si Sicong Li Sicoi Sicong Licng Siconcong Li S 

iter 100, loss: 30.965855
____
 a Yanhong Ni Lianhoni Yonheng Qaonhong QNYaihon Yug Lonhong Lonh Luaphanhong Qonheng Qaong Ng Long uang Ni Yang NaolYaChang Qonhong Qong HuChongMa Yanhong Huai Hang Hong aonheaMi ShonhangMa Yanhong Qa 

iter 200, loss: 28.877268
____
 heng Qi Yi Yi Li Li LongMang Li Li Li LongMuei Li Li Li LongMaei Long Lueneheng Qi Long Yuenueng Li Lon Yueieng Ni Yi Yi Li Lpeng Li Lon Z Li LongMNei Li Li Li Li Li Li Lig LongMa Yi Li Li LongMang Li 

iter 300, loss: 26.708679
____
 iwei Li Yiwei Li Li Yiweieng Li Li Yi  Ze Zeng Li Liweie

KeyboardInterrupt: 